In [2]:
# gmailer init


In [2]:
import datetime as dt

import yaml

import smtplib
import ssl
import mimetypes
from email.message import EmailMessage




# Email class

In [21]:
sender = "jaime.meriz13@gmail.com"
recipients = ["hexarunner@gmail.com"]
subject = "DEV" + "-" "Jmail" + "-" + str(dt.date.today())
body = "body\nyours\nDr.Lopez"
password = "password"

In [24]:
## password and config
config = "../config.yaml"
config = yaml.safe_load(open(config))

def parse_config(config: dict):
    """
    Parse the configuration yaml.

    Parameters
    -------
    config (dict{any}): Dictionary of congiruations.

    Returns
    -------
    credentials (dict(any)): Dictionary of credentials.
    """
    credentials = config["credentials"]
    ...
    return credentials

credentials = parse_config(config)

In [25]:

def send_mail(sender: str, recipients: list, subject:str, body: str, password: str):
    """
    Send an email via SMTP.

    Parameters
    -------
    sender (str): Sender as a string.
    recipients ([str]): List of recipients. 
    subject (str): Email subject line.
    body (str): Email body as string.
    password (str): Gmail password as string.
    
    Returns
    -------
    None
    """

    email = EmailMessage()
    email["Sender"] = sender
    email["Recipients"] = ", ".join(recipients)
    email["Subject"] = subject
    email.set_content(body)


    context = ssl.create_default_context()
    
    print("Sending email...")
   #  YAH >> Send this when connection is possible!
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp_server:
       smtp_server.login(sender, password)
       smtp_server.sendmail(sender, recipients, email.as_string())
    print("Message sent!")


    return email

password = credentials["gmail"]["password"]
email = send_mail(sender, recipients, subject, body, password)


Sending email...
Message sent!
